In [1]:
import torch
import torch.nn as nn
from torch import optim
#from torchdiffeq import odeint_adjoint as odeint
from scipy.linalg import expm
from torchdiffeq import odeint
import numpy as np
import csv

In [2]:
def dydt(y, t, A):
    return torch.mm(y,A)

In [3]:
def phi_A(y,A):
    return odeint(lambda t,x : dydt(x,t,A), y, torch.tensor([0., 1.]))[1]

In [4]:
class ODEFunc(torch.nn.Module):
    def __init__(self, A):
        super(ODEFunc, self).__init__()
        self.A = torch.nn.Parameter(torch.tensor(A))
        
    def forward(self, t, y):
        return dydt(y, t, self.A)

In [5]:
class NeuralODE(torch.nn.Module):
    def __init__(self, A_init):
        super(NeuralODE, self).__init__()
        self.func = ODEFunc(A_init)
        self.hidden_layer = torch.nn.Linear(2, 10)
        self.output_layer = torch.nn.Linear(10, 4)
        
    def forward(self, y):
        y = self.hidden_layer(y)
        y = torch.relu(y)
        y = self.output_layer(y)
        return y
    
    def get_A(self):
        return self.func.A

In [6]:
A_true = torch.tensor([[1., 2.], [3., 4.]])
training_losses={}
frob_losses={}

In [27]:
torch.diag(torch.Tensor([1,2,3,4]))

tensor([[1., 0., 0., 0.],
        [0., 2., 0., 0.],
        [0., 0., 3., 0.],
        [0., 0., 0., 4.]])

In [21]:
def train_model(model, x_data,y_data,epochs=500, lr=0.05):
    training_loss=[]
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.MSELoss()
    for epoch in range(epochs):
        optimizer.zero_grad()
        y_pred = odeint(model.func, x_data, torch.tensor([0., 1.]), method='dopri5')
        loss = criterion(y_pred, y_data)
        loss.backward()
        optimizer.step()
        if epoch % 20 == 0:
            print(f"Epoch {epoch}, Training Loss: {loss:.4f}")
            print(neural_ode.get_A())
            training_loss.append(int(loss.detach().numpy().item()))
    return training_loss

In [22]:
n_samples=500

x_data = torch.randn(n_samples, 2)
y_data = phi_A(x_data,A_true)

In [23]:
neural_ode = NeuralODE(torch.tensor([[0.8,2.3],[2.8,4.5]]))

training_loss=train_model(neural_ode, x_data,y_data)
A_estimated = neural_ode.get_A()

#ajout des résultats pour chaque n_samples
training_losses[n_samples]=training_loss
frob_losses[n_samples]=np.linalg.norm((A_true-A_estimated).detach().numpy())

print(f"Estimated A:\n{A_estimated}")

C:\Users\user\AppData\Local\Temp\ipykernel_21764\3190449401.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.A = torch.nn.Parameter(torch.tensor(A))


Epoch 0, Training Loss: 15510.5742
Parameter containing:
tensor([[0.7300, 2.2300],
        [2.7300, 4.4300]], requires_grad=True)
Epoch 20, Training Loss: 12397.1807
Parameter containing:
tensor([[0.7475, 2.0835],
        [2.7278, 4.2646]], requires_grad=True)
Epoch 40, Training Loss: 12279.7734
Parameter containing:
tensor([[0.8932, 1.9817],
        [2.8230, 4.1275]], requires_grad=True)
Epoch 60, Training Loss: 12256.6338
Parameter containing:
tensor([[1.0370, 1.9492],
        [2.9111, 4.0605]], requires_grad=True)
Epoch 80, Training Loss: 12253.8535
Parameter containing:
tensor([[1.0998, 1.9457],
        [2.9407, 4.0343]], requires_grad=True)
Epoch 100, Training Loss: 12253.8643
Parameter containing:
tensor([[1.1080, 1.9513],
        [2.9387, 4.0297]], requires_grad=True)
Epoch 120, Training Loss: 12253.8018
Parameter containing:
tensor([[1.1021, 1.9544],
        [2.9328, 4.0306]], requires_grad=True)
Epoch 140, Training Loss: 12253.7930
Parameter containing:
tensor([[1.0993, 1.9545

In [24]:
frob_losses

{500: 0.13084218}